# INFO 440 Final Project
<a href="https://colab.research.google.com/drive/1qzXe1g6NTdlua5eZ4KKLm4Qen7U0HAGH?usp=sharing">
        <img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg">
</a>
    <a href="https://www.python.org/">
            <img alt="Build" src="https://img.shields.io/badge/Made%20with-Python-1f425f.svg?color=purple">
</a>

## Extract Reddit Posts

In [ ]:
"""
File to extract the data from Reddit
"""
import datetime
import itertools
import pandas as pd
import praw
import re
import time

import matplotlib.pyplot as plt
import nltk
import seaborn as sns

from datetime import datetime, timedelta
from nltk.corpus import stopwords
from praw.models import MoreComments
from tqdm import tqdm

from collections import Counter
from nltk.util import ngrams
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer


SECRET = *****
DEV = *****
APP = *****
APP_ID = *****


def authenticate():
    """
    Function to authenticate a Reddit PRAW connection
    :return: _reddit, an authenticated Reddit instance
    """
    _reddit = praw.Reddit(client_id=APP_ID,
                          client_secret=SECRET,
                          user_agent=DEV)
    return _reddit


def extract_posts(_reddit, _subreddit_name, _time_filter):
    """
    Function to extract top 10,000 posts from ‘r/____’ from the past year and save to a file
    :param _reddit: an authenticated Reddit instance
    :param _subreddit_name: a string, name of the subreddit
    :return: _posts_df, dataframe of subreddit
    """
    _top_posts = []

    for post in tqdm(_reddit.subreddit(_subreddit_name).top(time_filter=_time_filter, limit=10000), desc="Extracting posts"):
        _top_posts.append(post)

        # Stop if we have enough posts
        if len(_top_posts) >= 10000:
            break

        #time.sleep(1)  # Sleep to avoid hitting rate limits

    # create a dataframe of the posts
    _posts_df = pd.DataFrame([vars(post) for post in _top_posts])

    print(f"Extracted {len(_posts_df)} posts.")
    print(f"DataFrame shape: {_posts_df.shape}")

    # Save the DataFrame to a CSV file
    _posts_df.to_csv(f"reddit_data/{_subreddit_name}.csv", index=False)

    return _posts_df


def get_distribution(_df):
    """
    Function to get the distribution of extracted data
    :param _df: posts dataframe
    """
    # Convert the 'created_utc' column to datetime format
    _df['created_utc'] = pd.to_datetime(_df['created_utc'], unit='s')

    # Print out all the dates
    print(_df['created_utc'])
    # _df['created_utc'].to_csv('dates_output.csv', index=False)

    # Plotting the date distribution
    plt.figure(figsize=(12, 6))
    sns.histplot(_df['created_utc'], bins=30, kde=True)  # kde=True adds a kernel density estimate
    plt.title('Date Distribution of Posts')
    plt.xlabel('Date')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Show the plot
    plt.show()

    earliest_date = _df['created_utc'].min()
    latest_date = _df['created_utc'].max()

    print(f"Earliest date: {earliest_date}")
    print(f"Latest date: {latest_date}")

In [ ]:
# authenticate a Reddit instance
reddit_instance = authenticate()

# get GameStop.csv dataframe
try:
    gamestop_posts_df = pd.read_csv("reddit_data/GameStop.csv")
except FileNotFoundError:
    gamestop_posts_df = extract_posts(reddit_instance, 'GameStop', _time_filter="year")
print(gamestop_posts_df.head())
print("GameStop.csv dataframe shape:", gamestop_posts_df.shape)
print("GameStop.csv dataframe columns:", gamestop_posts_df.columns)
get_distribution(gamestop_posts_df)

# get Tesla.csv dataframe
try:
    tesla_posts_df = pd.read_csv("reddit_data/Tesla.csv")
except FileNotFoundError:
    tesla_posts_df = extract_posts(reddit_instance, 'Tesla', _time_filter="all")
print(tesla_posts_df.head())
print("Tesla.csv dataframe shape:", tesla_posts_df.shape)
print("Tesla.csv dataframe columns:", tesla_posts_df.columns)
get_distribution(tesla_posts_df)

# get nvidia.csv dataframe
try:
    nvidia_posts_df = pd.read_csv("reddit_data/nvidia.csv")
except FileNotFoundError:
    nvidia_posts_df = extract_posts(reddit_instance, 'nvidia', _time_filter="year")
print(nvidia_posts_df.head())
print("nvidia.csv dataframe shape:", nvidia_posts_df.shape)
print("nvidia.csv dataframe columns:", nvidia_posts_df.columns)
get_distribution(nvidia_posts_df)

## Clean Extracted Data

In [ ]:
import pandas as pd
import re

In [ ]:
from google.colab import files

In [ ]:
gme = files.upload()

Saving GameStop.csv to GameStop (2).csv


In [ ]:
tesla = files.upload()

Saving Tesla.csv to Tesla (2).csv


In [ ]:
nvidia = files.upload()

Saving nvidia.csv to nvidia (2).csv


In [ ]:
# Load data
gamestop_posts_df = pd.read_csv('GameStop.csv')
tesla_posts_df = pd.read_csv('Tesla.csv')
nvidia_posts_df = pd.read_csv('nvidia.csv')

In [ ]:
gamestop_posts_df.head(1)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,num_crossposts,media,is_video,_fetched,_additional_fetch_params,_comments_by_id,is_gallery,media_metadata,gallery_data,author_cakeday
0,2048,confidence,<praw.reddit.Reddit object at 0x7fc3425d1640>,NaN,GameStop,NaN,t2_f2jqpy9n,False,NaN,0,...,0,NaN,False,False,{},{},NaN,NaN,NaN,NaN


In [ ]:
tesla_posts_df.head(1)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,media,is_video,_fetched,_additional_fetch_params,_comments_by_id,crosspost_parent_list,crosspost_parent,media_metadata,author_cakeday,link_flair_template_id
0,2048,confidence,<praw.reddit.Reddit object at 0x7fc3425d1640>,NaN,Tesla,NaN,t2_p0rbl,False,NaN,0,...,NaN,False,False,{},{},NaN,NaN,NaN,NaN,NaN


In [ ]:
nvidia_posts_df.head(1)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,media,is_video,_fetched,_additional_fetch_params,_comments_by_id,is_gallery,media_metadata,gallery_data,crosspost_parent_list,crosspost_parent
0,2048,confidence,<praw.reddit.Reddit object at 0x7ff44bde6a30>,NaN,nvidia,NaN,t2_10opls,False,NaN,0,...,NaN,False,False,{},{},NaN,NaN,NaN,NaN,NaN


In [ ]:
def create_relevant_df(df):
    """
    Function to create a df with relevant columns and merge 'selftext' and 'title' columns into one column
    :param df: posts dataframe
    :return: cleaned_df, a dataframe with relevant columns and a ‘text’ column (merged 'selftext' and 'title' columns into ‘text’ column)
    """
    # Extract the required columns from the original dataframe
    selected_columns = df[['score', 'created_utc', 'title', 'selftext']]

    # Combine 'title' and 'selftext' into a new 'text' column
    selected_columns['text'] = selected_columns['title'].fillna('') + ' ' + selected_columns['selftext'].fillna('')

    # Create a new dataframe with the 'score', 'date', and 'text' columns
    cleaned_df = selected_columns[['score', 'created_utc', 'text',]]

    return cleaned_df

In [ ]:
# Get gamestop relevant df
gme_cleaned_df = create_relevant_df(gamestop_posts_df)
gme_cleaned_df.head(2)

<ipython-input-39-1cb3c370b8ce>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['text'] = selected_columns['title'].fillna('') + ' ' + selected_columns['selftext'].fillna('')


,score,created_utc,text
0,1966,1.699123e+09,"Not my former store, but one 30 minutes from m..."
1,1933,1.698679e+09,Why even get a pro anymore? Soo no more renewa...


In [ ]:
# Get tesla relevant df
tesla_cleaned_df = create_relevant_df(tesla_posts_df)
tesla_cleaned_df.head(2)

<ipython-input-39-1cb3c370b8ce>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['text'] = selected_columns['title'].fillna('') + ' ' + selected_columns['selftext'].fillna('')


,score,created_utc,text
0,579,1.637509e+09,Tesla Statue on Cdn side of Niagara Falls 11/2...
1,447,1.625959e+09,It's Tesla's Birthday. Here's the stamps by Go...


In [ ]:
# Get nvidia relevant df
nvidia_cleaned_df = create_relevant_df(nvidia_posts_df)
nvidia_cleaned_df.head(2)

<ipython-input-39-1cb3c370b8ce>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['text'] = selected_columns['title'].fillna('') + ' ' + selected_columns['selftext'].fillna('')


,score,created_utc,text
0,3288,1.682701e+09,EA has released a statement on Star Wars Jedi:...
1,3009,1.683395e+09,GeForce 9500 GT in a Zotac RTX 4090 box. Oh ho...


In [ ]:
def clean_text(text):
    """
    Function to clean 'text' column
    :param text: strings in 'text' column of the df
    :return: text, cleaned strings for ‘text’ column
    """
    text = re.sub(r'<[^<]+?>', ' ', text)  # remove HTML tags
    text = text.lower() # Convert text to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'[^a-z\s]', '', text) # Remove special characters, numbers, and punctuation
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces

    return text

In [ ]:
# Clean text for gamestop
gme_cleaned_df['text'] = gme_cleaned_df['text'].apply(clean_text)
gme_cleaned_df.head(2)

,score,created_utc,text
0,1966,1.699123e+09,not my former store but one minutes from me th...
1,1933,1.698679e+09,why even get a pro anymore soo no more renewal...


In [ ]:
# Clean text for tesla
tesla_cleaned_df['text'] = tesla_cleaned_df['text'].apply(clean_text)
tesla_cleaned_df.head(2)

,score,created_utc,text
0,579,1.637509e+09,tesla statue on cdn side of niagara falls
1,447,1.625959e+09,its teslas birthday heres the stamps by govern...


In [ ]:
# Clean text for nvidia
nvidia_cleaned_df['text'] = nvidia_cleaned_df['text'].apply(clean_text)
nvidia_cleaned_df.head(2)

,score,created_utc,text
0,3288,1.682701e+09,ea has released a statement on star wars jedi ...
1,3009,1.683395e+09,geforce gt in a zotac rtx box oh how far weve ...


In [ ]:
gme_cleaned_df.shape

(990, 3)

In [ ]:
tesla_cleaned_df.shape

(534, 3)

In [ ]:
nvidia_cleaned_df.shape

(988, 3)

In [ ]:
'''
gme_cleaned_df.to_csv('gme_cleaned_past_year.csv', index=False)
tesla_cleaned_df.to_csv('tesla_cleaned_all_time.csv', index=False)
nvidia_cleaned_df.to_csv('nvidia_cleaned_past_year.csv', index=False)
'''

"\ngme_cleaned_df.to_csv('gme_cleaned_past_year.csv', index=False)\ntesla_cleaned_df.to_csv('tesla_cleaned_all_time.csv', index=False)\nnvidia_cleaned_df.to_csv('nvidia_cleaned_past_year.csv', index=False)\n"

## Sentiment Scoring for News using VADER

## Sentiment Scoring for News using FinBERT, BERTweet, and RoBERTa

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm
import warnings

def load_model_and_tokenizer(model_name):
    """Load the tokenizer and model for the given model name."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Suppress specific warning related to weights not being used
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Some weights of the model checkpoint at")
        model = AutoModelForSequenceClassification.from_pretrained(model_name)

    return tokenizer, model

def get_sentiment_score(tokenizer, model, text):
    """Calculate the sentiment score for a given text using the provided tokenizer and model."""
    # Truncate text properly using tokenizer's encoding capabilities
    inputs = tokenizer(text, truncation=True, max_length=512, return_tensors="pt")

    # Ensure inputs are on the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Perform the forward pass and handle potential issues
    with torch.no_grad():
        try:
            result = model(**inputs)
            logits = result.logits

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=1)

            # Extract probabilities for each sentiment
            positive_prob = probs[0][0].item()
            negative_prob = probs[0][1].item()

            # Calculate sentiment score in the range -1 to 1
            sentiment_score = positive_prob - negative_prob
        except IndexError as e:
            print(f"IndexError encountered for text: {text[:50]}... (truncated)")
            sentiment_score = 0  # Fallback to neutral score on error

    return sentiment_score

def apply_sentiment_analysis(input_file, output_file, model_configurations):
    """Apply sentiment analysis using multiple models and save the results to a CSV file."""
    # Load the dataset
    df = pd.read_csv(input_file)

    # Ensure all text data is string and handle missing values
    df['text'] = df['text'].astype(str).fillna('')

    # Loop over each model configuration, calculate sentiment scores, and add them as columns
    for config in model_configurations:
        model_name = config["model_name"]
        custom_name = config["custom_name"]

        # Load the tokenizer and model
        tokenizer, model = load_model_and_tokenizer(model_name)

        # Apply the sentiment analysis function to the dataframe with a progress bar
        tqdm.pandas(desc=f"Processing with {custom_name}")
        df[f'{custom_name}_sentiment_score'] = df['text'].progress_apply(lambda x: get_sentiment_score(tokenizer, model, x))

        print(f"Sentiment scores for model '{model_name}' have been labeled as '{custom_name}_sentiment_score'")

    # Save all sentiment scores in a single CSV file
    df.to_csv(output_file, index=False)
    print(f"All sentiment scores have been saved to {output_file}")


In [6]:
# Define the model configurations
model_configurations = [
    {"model_name": "yiyanghkust/finbert-tone", "custom_name": "FinBERT"},
    {"model_name": "finiteautomata/bertweet-base-sentiment-analysis", "custom_name": "BERTweet"},
    {"model_name": "cardiffnlp/twitter-roberta-base-sentiment-latest", "custom_name": "RoBERTa"}
]

# Run for GameStop data
input_file = './gme_cleaned_past_year.csv'
output_file = './gme_past_year_with_multiple_bert_sentiments.csv'
apply_sentiment_analysis(input_file, output_file, model_configurations)


Processing with FinBERT: 100%|██████████| 990/990 [08:14<00:00,  2.00it/s]


Sentiment scores for model 'yiyanghkust/finbert-tone' have been labeled as 'FinBERT_sentiment_score'


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Processing with RoBERTa: 100%|██████████| 990/990 [07:53<00:00,  2.09it/s]


Sentiment scores for model 'cardiffnlp/twitter-roberta-base-sentiment-latest' have been labeled as 'RoBERTa_sentiment_score'
All sentiment scores have been saved to ./gme_past_year_with_multiple_bert_sentiments.csv


In [9]:
# Run for Nvidia data
input_file = './nvidia_cleaned_past_year.csv'
output_file = './nvidia_past_year_with_multiple_bert_sentiments.csv'
apply_sentiment_analysis(input_file, output_file, model_configurations)

Processing with FinBERT: 100%|██████████| 988/988 [06:17<00:00,  2.62it/s]
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Sentiment scores for model 'yiyanghkust/finbert-tone' have been labeled as 'FinBERT_sentiment_score'


Processing with BERTweet:   1%|          | 6/988 [00:00<01:17, 12.72it/s]

IndexError encountered for text: giveaway gigabyte gaming oc from cablemod hello rn... (truncated)


Processing with BERTweet:  10%|█         | 99/988 [00:14<01:56,  7.66it/s]

...

IndexError encountered for text: is the a substantial enough upgrade from a super a... (truncated)


Processing with BERTweet: 100%|██████████| 988/988 [02:41<00:00,  6.13it/s]


Sentiment scores for model 'finiteautomata/bertweet-base-sentiment-analysis' have been labeled as 'BERTweet_sentiment_score'


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Processing with RoBERTa: 100%|██████████| 988/988 [06:06<00:00,  2.70it/s]

Sentiment scores for model 'cardiffnlp/twitter-roberta-base-sentiment-latest' have been labeled as 'RoBERTa_sentiment_score'
All sentiment scores have been saved to ./nvidia_past_year_with_multiple_bert_sentiments.csv


In [10]:
# Run for Tesla data
input_file = './tesla_cleaned_all_time.csv'
output_file = './tesla_all_time_with_multiple_bert_sentiments.csv'
apply_sentiment_analysis(input_file, output_file, model_configurations)

Processing with FinBERT: 100%|██████████| 534/534 [01:46<00:00,  5.03it/s]


Sentiment scores for model 'yiyanghkust/finbert-tone' have been labeled as 'FinBERT_sentiment_score'


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Processing with BERTweet:   3%|▎         | 16/534 [00:02<01:21,  6.38it/s]

IndexError encountered for text: im marc j seifer noted biographer of tesla who has... (truncated)


Processing with BERTweet:  28%|██▊       | 148/534 [00:19<00:36, 10.50it/s]

...

IndexError encountered for text: geek creation showcome support the tesla museum th... (truncated)


Processing with BERTweet: 100%|██████████| 534/534 [01:13<00:00,  7.23it/s]


Sentiment scores for model 'finiteautomata/bertweet-base-sentiment-analysis' have been labeled as 'BERTweet_sentiment_score'


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Processing with RoBERTa: 100%|██████████| 534/534 [01:33<00:00,  5.70it/s]

Sentiment scores for model 'cardiffnlp/twitter-roberta-base-sentiment-latest' have been labeled as 'RoBERTa_sentiment_score'
All sentiment scores have been saved to ./tesla_all_time_with_multiple_bert_sentiments.csv
